# fMRI Quality Control with MRIQC

Before we start to do anything with our data, we should check the acquisition quality. 

**MRIQC** extracts various [IQMs (image quality metrics)](https://mriqc.readthedocs.io/en/latest/measures.html) from structural (T1w and T2w) and functional MRI (magnetic resonance imaging) data.

See [MRIQC Documentation](https://mriqc.readthedocs.io/en/latest/).

MRIQC is a [BIDS-App](https://bids-apps.neuroimaging.io/), and therefore it inherently understands the BIDS standard and follows the BIDS-Apps standard command line interface: `mriqc bids-root/ output-folder/ participant`. BIDS-Apps are portable neuroimaging pipelines that understand BIDS datasets.


The best way to run MRIQC is using containerised versions (Docker or Singularity). Containers store the software and all of its dependencies (including a minimal operating system) in a single, large image so that when it comes time to run the software everything "just works". This makes the software both shareable and portable while the output becomes reproducible. At the CBU we have Singularity available. It is also available on HPC. Singularity can also be used with Docker images.

```bash
module load singularity
singularity pull --name mriqc_latest.sif docker://nipreps/mriqc:latest
```

## MRIQC Participant level

`step05_mriqc_run.sh`

```bash
#!/bin/bash
set -eu
# receive all directories, and index them per job array
SUBJECT_LIST=("${@:2}")
subject=${SUBJECT_LIST[${SLURM_ARRAY_TASK_ID}]}

# processing start time
start=$(date +%s)
date
echo Submitted subject: "${subject}"

# ======================================================================
# MRIQC with Singularity
# ======================================================================
singularity run --cleanenv -B "${1}":/"${1}" \
    /imaging/local/software/singularity_images/mriqc/mriqc-22.0.1.simg \
    "${1}"/data/bids \
    "${1}"/data/bids/derivatives/mriqc/ \
    --work-dir "${1}"/data/work/mriqc/"$subject" \
    participant \
    --participant-label "${subject#sub-}" \
    --float32 \
    --n_procs 16 --mem_gb 24 --ants-nthreads 16 \
    --modalities T1w bold \
    --no-sub
# attaching our project directory to the Singularity
# the Singularity file
# our BIDS data directory
# outut directory
# --work-dir: path where intermediate results should be stored
# analysis_level (participant or group)
# --participant-label: a list of participant identifiers
# --float32: cast the input data to float32 if it’s represented in higher precision (saves space and improves perfomance)
# --n_procs 16 --mem_gb 24 --ants-nthreads 16: options to handle performance
# --modalities: filter input dataset by MRI type
# --no-sub: turn off submission of anonymized quality metrics to MRIQC’s metrics repository
# ======================================================================

# procesing end time
end=$(date +%s)
echo Time elapsed: "$(TZ=UTC0 printf '%(%H:%M:%S)T\n' $((end - start)))"
```


`step05_mriqc_batch.sh`
```bash
#!/bin/bash
# ======================================================================
# Dace Apšvalka (MRC CBU 2022)
# ======================================================================
# set -eu

# project path
PROJECT_PATH=${1}
# where to output jobs
JOB_DIR="$PROJECT_PATH"/data/work/mriqc/jobs
mkdir -p "$JOB_DIR"
cd "$JOB_DIR" || exit

# get the subject list 
SUBJECT_DIRS=("$PROJECT_PATH"/data/bids/sub-*)
SUBJECT_LIST=("${SUBJECT_DIRS[@]##*/}") 

# submit to SLURM
sbatch --array=0-$((${#SUBJECT_LIST[@]} - 1)) "$PROJECT_PATH"/code/preprocessing/step05_mriqc_run.sh "${PROJECT_PATH}" "${SUBJECT_LIST[@]}"
```

To run my MRIQC batch job, I run this command in the terminal: `./step05_mriqc_batch.sh /imaging/correia/dace/training/summer-school/Example_v03/FaceRecognition`

Takes about 30-65 min

## MRIQC Group level

The 'Goup' level just aggregates subject level reports and links them together.

`step06_mriqc_group.sh`
```bash
#!/bin/bash
set -eu
# receive all directories, and index them per job array
SUBJECT_LIST=("${@:2}")
subject=${SUBJECT_LIST[${SLURM_ARRAY_TASK_ID}]}

# processing start time
start=$(date +%s)
date
echo Submitted subject: "${subject}"

singularity run --cleanenv -B "${1}":/"${1}" \
    /imaging/local/software/singularity_images/mriqc/mriqc-22.0.1.simg \
    "${1}"/data/bids "${1}"/data/bids/derivatives/mriqc/ \
    --work-dir "${1}"/data/work/mriqc/"$subject" \
    participant \
    --participant-label "${subject#sub-}" \
    --float32 \
    --n_procs 16 --mem_gb 24 \
    --ants-nthreads 16 \
    --modalities T1w bold \
    --ica \
    --no-sub

# procesing end time
end=$(date +%s)
echo Time elapsed: "$(TZ=UTC0 printf '%(%H:%M:%S)T\n' $((end - start)))"
```
`./step06_mriqc_group.sh /imaging/correia/dace/training/summer-school/Example_v03/FaceRecognition`

Takes about 17 seconds. 